# Installation
CNGI documentation is located here:
[https://cngi-prototype.readthedocs.io/en/latest/index.html](https://cngi-prototype.readthedocs.io/en/latest/index.html)

Google Colab requires specific older versions of some packages such as Pandas and Dask, so we will install CNGI without its normal dependencies and then manually install each dependency afterwards.

Normally, you would want to leave out the --no-dependencies option

In [1]:
import os, time
import numpy as np

start = time.time()
print("installing cngi (takes a few minutes)...")
os.system("apt-get install libgfortran3")
os.system("pip install --extra-index-url https://casa-pip.nrao.edu/repository/pypi-group/simple casatools")
os.system("pip install cngi-prototype==0.0.8 --no-dependencies")
os.system("pip install --upgrade dask")
os.system("pip install --upgrade xarray")
os.system("pip install --upgrade zarr")

elapsed = round(time.time()-start)
print(f'Finished installing after {elapsed} seconds')

print("downloading MeasurementSet from CASAguide First Look at Imaging...")
os.system("wget https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya_calibrated_flagged.ms.tar")
os.system("tar -xvf sis14_twhya_calibrated_flagged.ms.tar")
elapsed = round(time.time()-start)
print(f'Finished downloading after {elapsed} seconds')
print('complete')

installing cngi (takes a few minutes)...
Finished installing after 121 seconds
downloading MeasurementSet from CASAguide First Look at Imaging...
Finished downloading after 144 seconds
complete


# Initialize the Processing Environment
Colab [does not support](https://github.com/googlecolab/colabtools/issues/569) websocket connections between client and kernel, so scheduler status dashboard is inaccessible.
Possible to work around using ngrok?

In [2]:
from cngi.direct import InitializeFramework
client = InitializeFramework(workers=2,memory='6GB',processes=False)
client

/usr/local/lib/python3.6/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 99] Cannot assign requested address
  warnings.warn('\n' + msg)
/usr/local/lib/python3.6/dist-packages/distributed/deploy/local.py:197: UserWarning: 
Could not launch service 'bokeh' on port 8787. Got the following message:

[Errno 99] Cannot assign requested address
  self.scheduler.start(scheduler_address)


Client Scheduler: inproc://172.28.0.2/145/1,Cluster Workers: 2 Cores: 2 Memory: 12.00 GB


# Convert an MS to xarray NetCDF

In [3]:
from cngi.conversion import ms_to_ncdf

start = time.time()
ms_to_ncdf('sis14_twhya_calibrated_flagged.ms')
elapsed = round(time.time() - start)
print(f'Finished conversion in {elapsed} seconds')

processing ddi 0: chunks=1, size=53717
completed ddi 0
Complete.
Finished conversion in 25 seconds


# Open an xarray NetCDF based MS

(todo) Retrieve a summary of the xarray NetCDF MS file. 

Then create a new xarray Dataset from it.

This Dataset is the common data structure passed around to most other CNGI functions.

In [4]:
from cngi.ms import summarizeFile
from cngi.dio import read_ncdf

# returns summary as a pandas dataframe
#mssummary = summarizeFile('sis14_twhya_calibrated_flagged.pq')
#print(mssummary[['ddi','row_count_estimate','col_count','size_GB']])

# there is only one ddi in the MS, but pretend there are more and one is chosen
ddi = 0 #mssummary.ddi.values[0]

# here we create the dask dataframe for use in other CNGI functions
xds = read_ncdf('sis14_twhya_calibrated_flagged.ncdf',ddi=ddi)

# examine the start of the dataframe 
# note that the column selection should be made in to a convenience function
#cols = [col for col in ddf.columns.values if col not in list(ddf.columns.values[ddf.columns.str.match('(FLAG\d)|(R|IDATA\d)')])]
#ddf[cols].head()
xds

<xarray.Dataset>
Dimensions:         (chans: 384, pols: 2, rows: 80563, uvw: 3)
Coordinates:
  * pols            (pols) int32 0 1
  * chans           (chans) int32 0 1 2 3 4 5 6 ... 377 378 379 380 381 382 383
  * uvw             (uvw) int32 0 1 2
  * rows            (rows) int64 0 1 2 3 4 5 ... 80558 80559 80560 80561 80562
Data variables:
    ANTENNA1        (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    ANTENNA2        (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    ARRAY_ID        (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    DATA_DESC_ID    (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    EXPOSURE        (rows) float64 dask.array<chunksize=(53717,), meta=np.ndarray>
    FEED1           (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    FEED2           (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    FIELD_ID        (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    FLA

In [26]:
import xarray, zarr
xds.to_zarr('./measurement_set.zarr')

ValueError: ignored

In [0]:
xds = xarray.open_zarr('./measurement_set.zarr')

Perform a sequence of calculations defined by the original [dask demo notebook](https://colab.research.google.com/github/ryanraba/casa6/blob/master/casa7experiments.ipynb)
1. apply flags (sets flagged data cells to nan)
2. average magnitude of cross products
3. subtract mean magnitude of each baseline from visibilities
4. filter out baselines with outlier mean noise
5. take the mean accross channels to get a continuum
6. plot the UV space


In [0]:
original_real = xds.RDATA
original_imag = xds.IDATA

In [0]:
# 1. apply flags (sets flagged data cells to nan)
xds['RDATA'] = xds.RDATA.where(xds.FLAG.isin([True]), drop=False)
xds['IDATA'] = xds.IDATA.where(xds.FLAG.isin([True]), drop=False)

In [0]:
# 2. calculate mean values for each component of the complex visibilities
real_mean = xds.RDATA.mean(dim='chans', skipna=True)
imag_mean = xds.IDATA.mean(dim='chans', skipna=True)

Unfortunately, xarray hasn't implemented groupby operations with multiple arguments along the same axis - see this [stackoverflow post](https://stackoverflow.com/questions/37008103/python-xarray-grouping-by-multiple-parameters).

In [54]:
antennas = xds.groupby('ANTENNA1')
antennas.groups.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24])

In [37]:
baselines = xds.groupby(['ANTENNA1', 'ANTENNA2'])

TypeError: ignored

[MultiIndexing](https://github.com/pydata/xarray/issues/1603) could be a viable option....

In [58]:
print(xds.coords)
test = xds.assign_coords({'antenna1':np.asarray(list(xds.groupby('ANTENNA1').groups.keys())),
                          'antenna2':np.asarray(list(xds.groupby('ANTENNA2').groups.keys()))})
test.coords

Coordinates:
  * chans    (chans) int32 0 1 2 3 4 5 6 7 8 ... 376 377 378 379 380 381 382 383
  * pols     (pols) int32 0 1
  * rows     (rows) int64 0 1 2 3 4 5 6 ... 80557 80558 80559 80560 80561 80562
  * uvw      (uvw) int32 0 1 2


Coordinates:
  * chans     (chans) int32 0 1 2 3 4 5 6 7 ... 376 377 378 379 380 381 382 383
  * pols      (pols) int32 0 1
  * rows      (rows) int64 0 1 2 3 4 5 6 ... 80557 80558 80559 80560 80561 80562
  * uvw       (uvw) int32 0 1 2
  * antenna1  (antenna1) int64 1 2 3 4 5 6 7 9 11 ... 15 16 17 18 19 20 21 22 24
  * antenna2  (antenna2) int64 2 3 4 5 6 7 9 11 12 ... 17 18 19 20 21 22 24 25

In [69]:
test.set_index(baseline=['antenna1', 'antenna2'])

<xarray.Dataset>
Dimensions:         (baseline: 20, chans: 384, pols: 2, rows: 80563, uvw: 3)
Coordinates:
  * chans           (chans) int32 0 1 2 3 4 5 6 ... 377 378 379 380 381 382 383
  * pols            (pols) int32 0 1
  * rows            (rows) int64 0 1 2 3 4 5 ... 80558 80559 80560 80561 80562
  * uvw             (uvw) int32 0 1 2
  * baseline        (baseline) MultiIndex
  - antenna1        (baseline) int64 1 2 3 4 5 6 7 9 ... 16 17 18 19 20 21 22 24
  - antenna2        (baseline) int64 2 3 4 5 6 7 9 11 ... 18 19 20 21 22 24 25
Data variables:
    ANTENNA1        (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    ANTENNA2        (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    ARRAY_ID        (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    DATA_DESC_ID    (rows) int32 dask.array<chunksize=(53717,), meta=np.ndarray>
    EXPOSURE        (rows) float64 dask.array<chunksize=(53717,), meta=np.ndarray>
    FEED1           (rows) int32 d

In [67]:
test.groupby(['baseline'])

TypeError: ignored

Okay, so that didn't work. Perhaps it is possible to create a new DataArray that represents "baseline" (with same dimensions as other data variables) and group along that.

In [75]:
antennas = xds.groupby('ANTENNA1')
antennaz = xds.groupby('ANTENNA2')
antennas.dims

Frozen(SortedKeysDict({'rows': 7504, 'pols': 2, 'chans': 384, 'uvw': 3}))

Coordinates:
  * chans    (chans) int32 0 1 2 3 4 5 6 7 8 ... 376 377 378 379 380 381 382 383
  * pols     (pols) int32 0 1
  * rows     (rows) int64 0 1 2 3 4 5 6 ... 80557 80558 80559 80560 80561 80562
  * uvw      (uvw) int32 0 1 2


Coordinates:
  * chans     (chans) int32 0 1 2 3 4 5 6 7 ... 376 377 378 379 380 381 382 383
  * pols      (pols) int32 0 1
  * rows      (rows) int64 0 1 2 3 4 5 6 ... 80557 80558 80559 80560 80561 80562
  * uvw       (uvw) int32 0 1 2
  * antenna1  (antenna1) int64 1 2 3 4 5 6 7 9 11 ... 15 16 17 18 19 20 21 22 24
  * antenna2  (antenna2) int64 2 3 4 5 6 7 9 11 12 ... 17 18 19 20 21 22 24 25

In [41]:
da = xds.RDATA.set_index(exp_time=['experiment', 'time'])

Frozen(SortedKeysDict({'rows': 80563, 'pols': 2, 'chans': 384, 'uvw': 3}))

In [0]:
# 3. subtract mean magnitude of each baseline from visibilities
